In [1]:
# import dependencies
import gmaps
import pandas as pd

# Google developer API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [2]:
# import dataframe:
file="../WeatherPy/weather_API_output.csv"
weather_df=pd.read_csv(file)
weather_df

,Unnamed: 0,city_id,city,country,city_lat,city_lng,temp,humidity,cloudiness,wind_speed
0,0,1521230,Makīnsk,KZ,52.63,70.42,54.27,60,100,9.64
1,1,2964782,Dingle,IE,52.14,-10.27,62.60,88,40,4.70
2,2,3833367,Ushuaia,AR,-54.80,-68.30,39.20,80,75,8.05
3,3,964432,Port Alfred,ZA,-33.59,26.89,57.99,92,65,1.99
4,4,3372707,Ribeira Grande,PT,38.52,-28.70,68.00,68,40,16.11
...,...,...,...,...,...,...,...,...,...,...
872,872,3443061,Chui,UY,-33.70,-53.46,58.35,66,93,10.25
873,873,6167817,Torbay,CA,47.67,-52.73,46.89,100,90,4.70
874,874,2246451,Saint-Louis,SN,16.33,-15.00,98.60,32,1,2.24
875,875,4032243,Vaini,TO,-21.20,-175.20,69.80,68,20,12.75


In [18]:
# extract variables needed for gmaps
# Store latitude and longitude in locations
locations =weather_df[["city_lat", "city_lng"]]

# Fill NaN values and convert to float
humidity = weather_df["humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure(map_type="HYBRID", center=(20, 0), zoom_level=1.75)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# narrow down the weather_df to find my ideal weather condition.
# temp between 73 and 92 degrees
temp_filter=weather_df.loc[(weather_df["temp"]>73)&(weather_df["temp"]<91)]
# Humid enough that my skin won't dry out but not so humid that I feel like I am a steamed vegetable.
humidity_filter=temp_filter.loc[(temp_filter["humidity"]>40)&(temp_filter["humidity"]<50)]
# minimal cloudiness
cloudy_filter=humidity_filter.loc[humidity_filter["cloudiness"]<30]
# not too windy:
wind_filter=cloudy_filter.loc[cloudy_filter["wind_speed"]<10]
vacation_df=wind_filter.reset_index()
vacation_df=vacation_df.drop(["index"], axis=1) 
vacation_df

,Unnamed: 0,city_id,city,country,city_lat,city_lng,temp,humidity,cloudiness,wind_speed
0,75,3444105,San Joaquin,BO,-16.38,-58.39,87.37,42,26,4.41
1,376,3514277,Yecuatla,MX,19.87,-96.75,84.15,42,0,7.87
2,407,3665202,Aripuanã,BR,-9.17,-60.63,90.32,42,9,4.50
3,513,213940,Kasongo,CD,-4.45,26.67,78.12,45,0,3.36
4,660,1529376,Korla,CN,41.76,86.15,73.99,43,0,1.99
5,692,2620425,Aulum,DK,56.26,8.79,75.43,47,10,5.82
6,743,3470451,Barretos,BR,-20.56,-48.57,79.23,47,0,8.93


In [13]:
# * Using Google Places API to find the first hotel for each city located within 
# 5000 meters of your coordinates.
# find the closest restaurant of each type to coordinates

# install dependencies:
import requests
import json

# create empty dataframe columns in wind_filter:
vacation_df["hotel_name"]=""


# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set parameters values
target_radius = 5000
target_type = "lodging"

for index, row in vacation_df.iterrows():
    # get latitude & longitude
    lat= row["city_lat"]
    lon= row["city_lng"]
    city=row["city"]
    country=row["country"]
  
    # geocoordinates
    target_coordinates =f"{lat},{lon}"

    # set up a parameters dictionary
    params = {"location": target_coordinates,"radius": target_radius,"type": target_type,"key": gkey}
    
    # assemble url and make API request & jsonify response
    response = requests.get(base_url, params=params).json()
    
    #  print statement for when retrieving results:
    print(f"Retrieving Results for {city}, {country}, lat:{lat},long:{lon}")
    
    # print the response url, avoid doing for public github repos in order to avoid exposing key
    #print(response.url)
    try:
#         print(f"Closest hotel to {lat},{lon} is {response["results"][0]["name"]}.")
        
        vacation_df.loc[index, 'hotel_name'] = response["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for San Joaquin, BO, lat:-16.38,long:-58.39
------------
Retrieving Results for Yecuatla, MX, lat:19.87,long:-96.75
------------
Retrieving Results for Aripuanã, BR, lat:-9.17,long:-60.63
Missing field/result... skipping.
------------
Retrieving Results for Kasongo, CD, lat:-4.45,long:26.67
------------
Retrieving Results for Korla, CN, lat:41.76,long:86.15
------------
Retrieving Results for Aulum, DK, lat:56.26,long:8.79
------------
Retrieving Results for Barretos, BR, lat:-20.56,long:-48.57
------------


In [14]:
vacation_df

,Unnamed: 0,city_id,city,country,city_lat,city_lng,temp,humidity,cloudiness,wind_speed,hotel_name
0,75,3444105,San Joaquin,BO,-16.38,-58.39,87.37,42,26,4.41,"Hotel Balneario ""Flores"""
1,376,3514277,Yecuatla,MX,19.87,-96.75,84.15,42,0,7.87,La Esperanza Yecuatla Veracruz
2,407,3665202,Aripuanã,BR,-9.17,-60.63,90.32,42,9,4.50,
3,513,213940,Kasongo,CD,-4.45,26.67,78.12,45,0,3.36,Guest Hôtel
4,660,1529376,Korla,CN,41.76,86.15,73.99,43,0,1.99,Gulisitan Hotel
5,692,2620425,Aulum,DK,56.26,8.79,75.43,47,10,5.82,Hotel Aulum
6,743,3470451,Barretos,BR,-20.56,-48.57,79.23,47,0,8.93,Dan Inn Barretos


In [15]:
# create empty lists for coordinates and labels:
marker_locations=[]
marker_labels=[]

# fill the lists
for index,row in vacation_df.iterrows():
    marker_locations.append((row["city_lat"], row["city_lng"]))
    hotel=row["hotel_name"]
    city=row["city"]
    country=row["country"]
    marker_labels.append(f'{hotel}, {city}, {country}')

# create map    
fig=gmaps.figure(map_type="HYBRID", center=(20, 0), zoom_level=1.75)

# create markers layer:
markers=gmaps.marker_layer(marker_locations, info_box_content=marker_labels)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=1)

# Add layers
fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))